In [ ]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki_tool.description

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings



with open("最后一个问题.txt",encoding = 'UTF-8') as f:
    last_question = f.read()


text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([last_question])


db = Chroma.from_documents(texts, OpenAIEmbeddings())
retriever = db.as_retriever()
docs = retriever.get_relevant_documents("于是就有了光")
print(docs[0].page_content)

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "last_question_tool",
    "对于任何和科幻小说，阿西莫夫，小说最后一个问题等相关问题，都必须使用这个tool",
)

In [ ]:
tools = [wiki_tool, retriever_tool]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:

agent_executor.invoke({"input": "hello!"})

In [ ]:
agent_executor.invoke({"input": "科幻小说，要有光"})

In [ ]:
agent_executor.invoke({"input": "介绍下微软公司"})